In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# Paths
SYNTHETIC_DATA_DIR = r"C:\IMMAGE ANALYSIS PROJECT DATASET\weed\weeds\output\images"
MASK_DIR = r"C:\IMMAGE ANALYSIS PROJECT DATASET\weed\weeds\output\masks"
VISUALIZATION_DIR = r"C:\IMMAGE ANALYSIS PROJECT DATASET\weed\weeds\output\visualization"


class LightweightUNet(nn.Module):
    def __init__(self, n_classes=3):
        super(LightweightUNet, self).__init__()
        
        # Encoder (downsampling)
        self.enc1 = self._conv_block(3, 32)
        self.enc2 = self._conv_block(32, 64)
        self.enc3 = self._conv_block(64, 128)
        self.enc4 = self._conv_block(128, 256)
        
        # Decoder (upsampling)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = self._conv_block(256, 128)
        
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = self._conv_block(128, 64)
        
        self.up1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec1 = self._conv_block(64, 32)
        
        self.final = nn.Conv2d(32, n_classes, kernel_size=1)
        self.max_pool = nn.MaxPool2d(2)
        
    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        # Encoder path
        e1 = self.enc1(x)
        p1 = self.max_pool(e1)
        
        e2 = self.enc2(p1)
        p2 = self.max_pool(e2)
        
        e3 = self.enc3(p2)
        p3 = self.max_pool(e3)
        
        # Bridge
        e4 = self.enc4(p3)
        
        # Decoder path
        d3 = self.up3(e4)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)
        
        d2 = self.up2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)
        
        d1 = self.up1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)
        
        return self.final(d1)

class PlantSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [img for img in sorted(os.listdir(image_dir)) if img.endswith(('.png', '.jpg', '.jpeg'))]
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('synthetic', 'mask'))
        
        try:
            # Load image
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Image not found or unable to read at {img_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Load mask
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                raise ValueError(f"Mask not found or unable to read at {mask_path}")
            
            # One-hot encode mask
            mask_one_hot = np.zeros((3, mask.shape[0], mask.shape[1]), dtype=np.float32)
            for i in range(3):
                mask_one_hot[i, :, :] = (mask == i).astype(np.float32)

            # Apply transformations
            if self.transform:
                augmented = self.transform(image=image, mask=mask_one_hot.transpose(1, 2, 0))
                image = augmented['image']
                mask_one_hot = augmented['mask'].permute(2, 0, 1)  # Change from HWC to CHW format

            return image, mask_one_hot
        
        except Exception as e:
            print(f"Error loading file {img_name}: {e}")
            raise

def get_transforms(train=True):
    if train:
        return A.Compose([
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.GaussNoise(p=0.2),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

def plot_training_history(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training History')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    with tqdm(dataloader, desc='Training') as pbar:
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    
    return total_loss / len(dataloader)

def main():
    # Set up paths and parameters
    BATCH_SIZE = 4  # Reduced batch size
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Datasets and loaders
    train_dataset = PlantSegmentationDataset(
        SYNTHETIC_DATA_DIR,
        MASK_DIR,
        transform=get_transforms(train=True)
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,  # No multiprocessing
        pin_memory=True
    )
    
    # Initialize model, criterion, optimizer
    model = LightweightUNet().to(DEVICE)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training loop
    train_losses = []
    
    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        
        # Train
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
        train_losses.append(train_loss)
        
        print(f"Train Loss: {train_loss:.4f}")
    
    # Plot and save training history
    plot_training_history(train_losses, [])
    print("Training completed!")

if __name__ == "__main__":
    main()


In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import os
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score

# Define the paths and device
model_dir = "saved_models"
test_image_dir = "path_to_test_images"
test_mask_dir = "path_to_test_masks"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the metric calculations
def calculate_metrics(pred, target):
    pred = pred.detach().cpu().numpy()
    target = target.detach().cpu().numpy()
    
    # Threshold the predictions
    pred = (pred > 0.5).astype(np.uint8)
    
    iou = jaccard_score(target.flatten(), pred.flatten(), average='macro')
    dice = f1_score(target.flatten(), pred.flatten(), average='macro')
    precision = precision_score(target.flatten(), pred.flatten(), average='macro')
    recall = recall_score(target.flatten(), pred.flatten(), average='macro')
    
    return iou, dice, precision, recall

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    iou_total, dice_total, precision_total, recall_total = 0, 0, 0, 0
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            iou, dice, precision, recall = calculate_metrics(outputs, masks)
            
            iou_total += iou
            dice_total += dice
            precision_total += precision
            recall_total += recall
    
    num_batches = len(dataloader)
    return (iou_total / num_batches, dice_total / num_batches,
            precision_total / num_batches, recall_total / num_batches)

# Load test dataset
test_transform = get_transforms(train=False)
test_dataset = PlantSegmentationDataset(test_image_dir, test_mask_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Loop through all saved models and evaluate
results = {}
for model_file in sorted(os.listdir(model_dir)):
    if model_file.endswith('.pt'):
        model_path = os.path.join(model_dir, model_file)
        print(f"Evaluating {model_file}...")
        
        # Load model
        model = LightweightUNet().to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        
        # Calculate metrics
        iou, dice, precision, recall = evaluate_model(model, test_loader)
        results[model_file] = {"IoU": iou, "Dice": dice, "Precision": precision, "Recall": recall}

# Print the results for storytelling
print("Evaluation results for each model checkpoint:")
for model_file, metrics in results.items():
    print(f"{model_file} - IoU: {metrics['IoU']:.4f}, Dice: {metrics['Dice']:.4f}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}")

# Insights: Identify the highest scoring models to help decide on which checkpoint(s) perform best.
best_model = max(results, key=lambda x: results[x]['IoU'])
print(f"\nBest model based on IoU: {best_model} with IoU: {results[best_model]['IoU']:.4f}")

# Optionally, save the results to a file for further analysis.
import json
with open("model_performance_summary.json", "w") as f:
    json.dump(results, f, indent=4)


NameError: name 'get_transforms' is not defined